In [1]:
import json
import pandas as pd
import numpy as np
import re

In [2]:
with open("../data/imdb_minfo.txt", 'r') as f:
    imdb = json.load(f)

In [3]:
with open('../data/yelp_minfo.txt', 'r') as g:
    yelp = json.load(g)

In [4]:
df_yelp = pd.DataFrame(list(yelp.items()), columns=['feature', 'yelp_IG'])
df_imdb = pd.DataFrame(list(imdb.items()), columns=['feature', 'imdb_IG'])

In [5]:
print(df_yelp.shape, df_imdb.shape)

(5735, 2) (6661, 2)


In [6]:
df_yelp.head()

,feature,yelp_IG
0,negate,0.073813
1,punc_count,0.020343
2,percent_stop,0.023974
3,word_count,0.033897
4,exclaim_count,0.027289


In [7]:
df_final = pd.merge(df_yelp, df_imdb, how='outer', on='feature')

In [8]:
for i,r in df_final.iterrows():
    df_final.at[i, 'abs_diff'] = abs(df_final.at[i, 'yelp_IG'] - df_final.at[i, 'imdb_IG'])
    


In [9]:
df_final.head(10)

,feature,yelp_IG,imdb_IG,abs_diff
0,negate,0.073813,0.006257,0.067556
1,punc_count,0.020343,0.007457,0.012887
2,percent_stop,0.023974,0.001224,0.022750
3,word_count,0.033897,0.018275,0.015623
4,exclaim_count,0.027289,0.001155,0.026134
5,avg_sent,0.069084,0.110686,0.041602
6,"""_grams",0.105753,0.317433,0.211681
7,$_grams,0.097437,NaN,NaN
8,$1_grams,0.039000,NaN,NaN
9,(_grams,0.155949,0.345544,0.189595


In [10]:
df2 = df_final.sort_values('yelp_IG', ascending=False)

In [11]:
df3 = df2[(df2['yelp_IG']!=0.0)]

In [12]:
df3.shape

(6060, 4)

In [13]:
std = np.std(df3['abs_diff'])
mean = np.mean(df3['abs_diff'])
print(std)
print(mean)
print(mean+(std/2))

0.058773085342174305
0.038654644063172754
0.0680411867342599


In [14]:
df3.abs_diff.quantile(.75)

0.062094383260806144

In [15]:
df3 = df3[df3['yelp_IG'] >0.001]
df3.shape

(2781, 4)

In [16]:
df3 = df3[df3['imdb_IG'] >0.001]
df3.shape

(2674, 4)

In [17]:
df3 = df3[df3['abs_diff'] < 0.068]
df3.shape

(1505, 4)

In [18]:
df3 = df3.sort_values('yelp_IG', ascending=False)

nosuffix = df3[~df3['feature'].str.contains('_')]
suffix = df3[df3['feature'].str.contains('_')]

In [19]:
feature_set = suffix['feature'].tolist()

In [20]:
final_vocab = []
for i in feature_set:
    i = re.sub('\_grams$', '', i)
    i = re.sub('\_pos$', '', i)
    i = re.sub('\_neg$', '', i)
    final_vocab.append(i)


final_vocab = pd.DataFrame(final_vocab, columns=['feat'])

In [21]:
feat_list = final_vocab[~final_vocab['feat'].str.contains('_')].reset_index(drop=True)
feat_list = feat_list['feat'].tolist()
other_feat = final_vocab[final_vocab['feat'].str.contains('_')].reset_index(drop=True)
other_feat = other_feat['feat'].tolist()
other_feat.append('negate')
other_feat = pd.DataFrame(other_feat)

In [22]:
gram_lizt = []
word_lizt = []
for i in feat_list:
    if len(i)==2:
        gram_lizt.append(i)
    elif len(i)==3:
        gram_lizt.append(i)
    else:
        word_lizt.append(i)

In [23]:
def setdiff_sorted(array1,array2,assume_unique=False):
    ans = np.setdiff1d(array1,array2,assume_unique).tolist()
    if assume_unique:
        return sorted(ans)
    return ans

In [24]:
gram_list = setdiff_sorted(gram_lizt,word_lizt,assume_unique=False)
len(gram_list)

912

In [25]:
gram_list = pd.DataFrame(gram_lizt, columns=['feat'])
gram_list.to_csv('../data/gram_list.csv',index=False)
word_list = pd.DataFrame(word_lizt, columns=['feat'])
word_list.to_csv('../data//word_list.csv',index=False)
other_feat.to_csv('../data/other_feat.csv',index=False)

In [26]:
len(word_list) 

577

In [27]:
other_feat

,0
0,avg_sent
1,word_count
2,exclaim_count
3,percent_stop
4,punc_count
5,negate
